# 파이썬스러운(Pythonic)

## 인덱싱과 슬라이싱

In [1]:
my_list = [4,3,8,5,2,1,9,0]

In [2]:
# indexing
my_list[3] # 3번째요소 -> 5

5

In [3]:
my_list[0] # 0번째요소 -> 4

4

In [4]:
my_list[-1] # 마지막요소 -> 0

0

위와 같이 파이썬에서 인덱싱을 할때에 음수인덱스를 이용할 수 있다.   
즉, **의미적으로** 마지막요소를 나타내거나, 뒤에서 두번째요소 등을 나타낼 때에는 음수인덱스를 사용하자.

In [6]:
# slicing
my_list[1:3] # 1번째 요소부터 2번째요소 -> [3,8]

[3, 8]

In [7]:
my_list[4:] # 4번째 요소부터 끝까지 -> [2,1,9,0]

[2, 1, 9, 0]

In [10]:
# slice 객체 직접 빌드해서 전달하기 => slice(시작,중지,간격)
interval = slice(1,4)
my_list[interval]

[3, 8, 5]

In [11]:
interval = slice(1,7,2)
my_list[interval]

[3, 5, 1]

위와 같은 기능들은 \_\_getitem\_\_ 라는 매직 메서드 덕분에 동작하는 것이다.   
이것은 myobject[key] 와 같은 형태를 사용할 때 호출되는 메서드로 key에 해당하는 대괄호 안의 값을 파라미터로 전달한다.

### 시퀀스 객체 만들기
클래스가 리스트(또는 표준 라이브러리)의 래퍼인 경우 리스트의 동일한 메서드를 호출하여 호환성을 유지할 수 있다.

In [12]:
class items:
    def __init__(self, *values):
        self._values = list(values)
        
    def __len__(self):
        return len(self._values)
    
    def __getitem__(self, item):
        return self._values.__getitem__(item)

만약, 래퍼도 아니고 내장 객체를 사용하지도 않는 경우는 자신만의 시퀀스를 직접 구현할 수 있다. 이때에 다음과 같은 사항을 유의해야 한다.   
* 범위로 인덱싱하는 결과는 해당 클래스와 같은 타입의 인스턴스여야 한다.
* slice에 의해 제공된 범위는 파이썬이 하는 것처럼 마지막 요소는 제외해야 한다.

## 컨텍스트 관리자(context manager)
주요 동작의 전후에 작업을 실행하려고 할 때 유용하다.   
일반적으로는 리소스 관리와 관련하여 컨텍스트 관리자를 자주 볼 수 있다.(with 절)   

```
fd = open(filename)
try:
    process_file(fd)
finally:
    fd.close() # resource close
```

하지만 위와 같은 리소스 관리를 파이썬스럽게 구현하면 다음과 같다.

```
with open(filename) as fd:
    process_file(fd)
```

with문(PEP-343)은 컨텍스트 관리자로 진입하게 된다.   
이 경우 open 함수는 컨텍스트 관리자 프로토콜을 구현한다. 즉, 예외가 발생한 경우에도 블록이 완료되면 파일이 자동으로 닫힌다.   
사실, '자동'이라는 의미보다 블록이 종료되는 시점에 \_\_exit\_\_이라는 매직 메서드를 호출하는 것이다.

즉, 컨텍스트 관리자는 \_\_enter\_\_ 와 \_\_exit\_\_라는 두 개의 매직 메서드로 구성되는 것이다.   

### 컨텍스트 관리자 구현하기(1) - 제네레이터 사용

컨텍스트 관리자를 구현하는데에 있어서 contextmanager라는 데코레이터를 이용한다.   
contextmanager 데코레이터를 함수에 적용하면, 해당 함수의 코드는 컨텍스트 관리자로 변환한다.   
이때 함수는 **제네레이터**라는 특수한 형태이어야 한다. 이 함수는 코드의 문장을 \_\_enter\_\_와 \_\_exit\_\_로 분리한다.

In [14]:
import contextlib

def stop_database():
    print(f"database stop")
    
def start_database():
    print(f"database start")    
    
def db_backup():
    print(f"db backup - - - 1 ")
    print(f"db backup - - - 2 ")
    print(f"db backup - - - 3 ")

@contextlib.contextmanager
def db_handler():
    stop_database()
    yield
    start_database()
    
with db_handler():
    db_backup()
    


database stop
db backup - - - 1 
db backup - - - 2 
db backup - - - 3 
database start


위와 같이 db_handler() 라는 제네레이터 함수를 적용하고 해당 함수에 @contextlib.contextmanager 데코레이터를 적용하였다.   
(함수는 yield문을 사용했으므로 제네레이터 함수가 된다.)   
실제로 실행 결과를 보면, yield문 이전의 stop_database() 함수가 \_\_enter\_\_ 매직메서드 역할을 하였으며, yield문 이후의 start_database()함수가 \_\_exit\_\_ 매직메서드 역할을 한것을 볼 수 있다.

### 컨텍스트 관리자 구현하기(2) - 데코레이터 활용하기
이번에도 동일한 contextlib 라이브러리를 활용하는데, 그 중에서 ContextDecorator라는 클래스를 이용한다.   
우선 코드를 살펴보자

In [17]:
class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()
    
    def __exit__(self, ext_type, ex_value, ex_traceback):
        start_database()
        
@dbhandler_decorator()
def offline_backup():
    print("offline_backup - - - 1")
    print("offline_backup - - - 2")
    print("offline_backup - - - 3")
    
offline_backup()

database stop
offline_backup - - - 1
offline_backup - - - 2
offline_backup - - - 3
database start


이전 방법과 다른 점은, with문이 없다는 점이다. 단순히 decorator를 적용한 함수를 실행하면 해당 함수가 컨텍스트 관리자 안에서 실행된다.   
이것은 컨텍스트 관리자가 완전히 독립적이라는 점에서 좋을 수 있다. 하지만 컨텍스트 관리자 내부에서 사용하고자 하는 객체를 함수에서 받아서 사용할 수 없다.   
만약 메서드가 반환한 객체를 할당하여 사용하려면 이전의 방법을 사용해야 한다.   
또 다른 이점으로는, 위와 같은 데코레이터를 하나 만들어두면 중복해서 재사용할 수 있다는 점이다.   
